In [70]:
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [71]:
def map_labels(x):
    if x == 'important' or x == '\'important\'' or x == '1' or x == '\'1\'':
        return 1
    elif x == 'not important' or x == '\'not important\'' or x == '0' or x == '\'0\'':
        return 0
    return x

In [72]:
aya_df = pd.read_csv('../Datasets/System_User_Prompt_Aya_Numbered_Results.csv')
# columns_to_evaluate = ['predicted_k_0', 'predicted_k_1', 'predicted_k_5', 'predicted_k_20', 'predicted_k_50']
columns_to_evaluate = ['predicted_k_20']

In [73]:
aya_df.head(5)

,text,text_type,real_tag,predicted_k_0,predicted_k_1,predicted_k_2,predicted_k_3,predicted_k_4,predicted_k_5,predicted_k_10,predicted_k_20,predicted_k_50
0,واکنش کنسولگری ایران در استانبول به ریجکت شدن ...,only_title,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,برگزاری دادگاه پرونده کثیرالشاکی شرکت کاغذی «آ...,only_title,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,خبر جدید وزیر بهداشت درباره بازگشایی مدارس در ...,only_title,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,شکایت باشگاه استقلال از عیسی آل کثیر,only_title,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,دولت فرانسه مسئول عواقب اهانت بی‌شرمانه علیه م...,only_title,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [74]:
aya_df['real_tag'] = aya_df['real_tag'].apply(map_labels)
for column in columns_to_evaluate:
    aya_df[column] = aya_df[column].apply(map_labels)

In [75]:
true_labels = aya_df['real_tag'].dropna()
true_labels_first_400 = aya_df['real_tag'].dropna().iloc[:401]

num_true1 = (true_labels_first_400 == 1).sum()
num_true0 = (true_labels_first_400 == 0).sum()

# num_true1 = (true_labels == 1).sum()
# num_true0 = (true_labels == 0).sum()

print(f"  Number of true '1' labels: {num_true1}")
print(f"  Number of true '0' labels: {num_true0}")

  Number of true '1' labels: 77
  Number of true '0' labels: 324


In [76]:
for column in columns_to_evaluate:

    pred_labels = aya_df[column].dropna()

    common_indices = true_labels.index.intersection(pred_labels.index)

    true_labels = true_labels.loc[common_indices]
    pred_labels = pred_labels.loc[common_indices]

    # for label in pred_labels:
    #     print(label)

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    num_58 = (aya_df[column] == 1).sum()
    num_47 = (aya_df[column] == 0).sum()

    print(f"Metrics for column {column}:")
    print(f"  Accuracy: {accuracy:.2f}")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1 Score: {f1:.2f}")
    print(f"  Number of '58' labels: {num_58}")
    print(f"  Number of '47' labels: {num_47}")
    print("\n")


Metrics for column predicted_k_20:
  Accuracy: 0.80
  Precision: 0.48
  Recall: 0.43
  F1 Score: 0.45
  Number of '58' labels: 69
  Number of '47' labels: 332




In [77]:
for column in columns_to_evaluate:

    pred_labels = aya_df[column].dropna()

    common_indices = true_labels.index.intersection(pred_labels.index)
    true_labels_common = true_labels.loc[common_indices]
    pred_labels_common = pred_labels.loc[common_indices]

    print(f"Metrics for column {column}:")

    report = classification_report(true_labels_common, pred_labels_common, target_names=['0', '1'])
    print(report)

    num_1 = (aya_df[column] == 1).sum()
    num_0 = (aya_df[column] == 0).sum()

    print(f"  Number of '1' labels: {num_58}")
    print(f"  Number of '0' labels: {num_47}")
    print("\n")

Metrics for column predicted_k_20:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       324
           1       0.48      0.43      0.45        77

    accuracy                           0.80       401
   macro avg       0.67      0.66      0.67       401
weighted avg       0.79      0.80      0.80       401

  Number of '1' labels: 69
  Number of '0' labels: 332


